In [1]:
import os
import tvm
import nnvm
import topi

import nnvm.symbol as sb
from tvm.contrib import graph_runtime

import numpy as np
import tensorflow as tf

EXAMPLE = 1
target, target_host = 'llvm', 'llvm'
ctx = tvm.cpu(0)

os.environ['XDNN_VERBOSE'] = "1"

In [2]:
import xdnn


XDNN library .so file /workspace/MLsuite/xfdnn/rt/xdnn_cpp/lib/libxfdnn.so.2017.4 not found


In [3]:
platform = 'alveo-u200'

config = {
    'platform': platform,
    'xclbin': "../../../../MLsuite/overlaybins/" + platform + "/overlay_3.xclbin",
    'memory': 5,
    'dsp': 56,
    'netcfg': "work/tvm_compiler.json",
    #'fromtensorflow': False,
    'weights_name': "weights", 
    #datadir': "work/weights_data",
    'quantizecfg': "", # Empty for now to test if quantization is needed in this simple example
    'pngfile': "graph.png",
    'verbose': True
}

xdnn.xdnn_frontend.init(
    platform=config['platform'],
    xclbin=config['xclbin'],
    memory=config['memory'],
    dsp=config['dsp'],
    netcfg=config['netcfg'],
    weights_name=config['weights_name'],
    input_shape=(1,1,4,4),
    pngfile=config['pngfile'],
    verbose=config['verbose']
)

if EXAMPLE == 1:
    input_shape = (1,1,4,4)
    input_type = 'float32'
    output_shape = (1,1,2,2)
    
    data = np.reshape(np.array([[1,1,0,4],[5,1,0,8],
                                [3,5,1,0],[1,9,3,4]]), input_shape)
    x = sb.Variable("x")
    z = sb.max_pool2d(x, pool_size=[2,2], strides=[2,2])
    graph = nnvm.graph.create(z)
    print(graph.index.nodes)
    
    print("\nNNVM COMPILER BUILD\n")
    
    params = {}
    shape_dict = { 'x': input_shape }
    dtype_dict = { 'x': input_type }
    graph, lib, params = nnvm.compiler.build(
                graph, target, shape_dict, dtype_dict,
                params=params, target_host=target_host)
    
    print(graph.index.nodes)
    print(lib)
    print(params)

    print("\nXDNN SETUP FPGA EXECUTER\n")
    # xdnn.xdnn_frontend.setup_fpga_executer()

    print("\nXDNN CREATE GRAPH RUNTIME\n")
    #m = graph_runtime.create(graph, lib, ctx)
    #m.set_input('x', tvm.nd.array(data.astype(input_type)))

    #m.run()

    #tvm_output = m.get_output(0, tvm.nd.empty(((1, 1, 2, 2)), 'float32'))

    #print(tvm_output)
    
elif EXAMPLE == 2:
    pass

networkfile None
memory 5
dsp 56
generatefile work/tmp_tvm_compiler
fromtensorflow False
weights weights
bytesperpixels 1
cpulayermustgo True
pngfile graph.png
verbose True
Namespace(anew=None, approximate=False, banditpre=None, barrier=False, bridges=None, bytesperpixels=1, concatstrategy=None, conv_1x1_s2=False, cpulayermustgo=True, customreplication=None, customtiling=None, cut=None, ddr=256, dedicateddsp=None, dsp=56, fancyreplications=False, fc=False, finalnode=None, forceweights=None, forceweightsfullyconnected=False, fromtensorflow=False, frontendonly=False, generatefile='work/tmp_tvm_compiler', initialnode=None, inputcut=None, laodschedule=None, lasttensorbyname=None, leavescalealone=False, loadpickle=None, manasadebugmode=None, manualbatch=False, manualdeconv=False, memory=5, mixmemorystrategy=False, networkfile='None', noconvexity=False, nodynamicscaling=False, noreplication=False, notcaffeanew=False, parallelism=False, parallelismgraphalgorithm='tfs', parallelismstrategy=Non

input0 -> input0_blob  [label="input0->input0_blob"];
input0_blob -> max_pool2d0  [label="input0_blob->max_pool2d0"];
max_pool2d0 -> max_pool2d0_blob  [label="max_pool2d0->max_pool2d0_blob"];
max_pool2d0_blob -> max_pool2d0_output  [label="max_pool2d0->max_pool2d0_output"];
max_pool2d0_output -> max_pool2d0_output_blob  [label="max_pool2d0_output->max_pool2d0_output_blob"];
MAX  1
TOP 5
__________
1 ['max_pool2d0'] size:768 remap:[] data movement:[]
1	input0_blob M[0,512] Z=512 F=[1] B=[0] E=[] S=['layer'] [] L=-1 T=SizeType(batches=1, channels=1, height=4, width=4)
1	max_pool2d0_blob M[0,256] Z=256 F=[2] B=[1] E=[] S=['layer'] [] L=-1 T=SizeType(batches=1, channels=1, height=2, width=2)
__________
0 ['input0'] size:512 remap:[] data movement:[]
0	input0_blob M[0,512] Z=512 F=[1] B=[0] E=[] S=['layer'] [] L=-1 T=SizeType(batches=1, channels=1, height=4, width=4)
__________
2 ['max_pool2d0_output'] size:512 remap:[] data movement:[]
2	max_pool2d0_output_blob M[0,256] Z=256 F=[] B=[2] E=

Encoding {'XNConv': 'id XNOp name kernel_w kernel_h strides_w strides_h padding_w padding_h dilation_w dilation_h preshift scale postshift relu bias inaddr insize_w insize_h inchan outaddr outsize_w outsize_h  outchan Bypass_Perf_Opt ', 'XNDeconv': 'id XNOp name kernel_w kernel_h strides_w strides_h padding_w padding_h dilation_w dilation_h preshift scale postshift relu bias inaddr insize_w insize_h inchan outaddr outsize_w outsize_h  outchan Bypass_Perf_Opt ', 'XNConvP': 'id XNOp name kernel_w kernel_h strides_w strides_h padding_w padding_h dilation_w dilation_h preshift scale postshift relu bias inaddr insize_w insize_h inchan outaddr outsize_w outsize_h  outchan Bypass_Perf_Opt  pool_kernel_w pool_kernel_h pool_strides_w pool_strides_h pool_paddings_w pool_paddings_h pool_fcmode pool_inaddr pool_insize_w pool_insize_h pool_inchan pool_outaddr pool_outsize_w pool_outsize_h', 'XNUpload': 'id XNOp inaddr insize inchan', 'XNInner': 'id XNOp name relu prelu preshift scale postshift matr

Processing weights for 3 schedule steps: 1We have Data to write  input0 MemoryRequirements(size=512, input=[], output=[512, [512]], internal=[2, []], extra=None, strategy=[], layout=-1, timestamp=-1, shapes=None, replication=None, written=False, specifier='', IO=False) 0
We DON'T write  ParametersLayer(type=['Input'], number_outputs=None, paddings=None, kernel_sizes=None, strides=None, dilation=None, group=None, global_pooling=None, shapes=SizeType(batches=1, channels=1, height=4, width=4), sizes=MemoryRequirements(size=512, input=[], output=[512, [512]], internal=[2, []], extra=None, strategy=[], layout=-1, timestamp=-1, shapes=None, replication=None, written=False, specifier='', IO=False), quantizations=None, batches=None, layer_type=['layer'], extras_and_future=None, tops=['input0'], bottoms=[], layer=[], dag=ColorForDAG(active=[2], schedule=0, forward=[1], backward=[], extra=None, hook=[]), systolic_width=56, word_width=64, alignedsizes=None, bias=None, scale=None, preshift=None, p

In [ ]:
import os
import xfdnn.rt.xdnn as xdnn_lib

os.environ['XDNN_VERBOSE'] = "1"

ret, handles = xdnn_lib.createHandle("../../../../MLsuite/overlaybins/alveo-u200/overlay_3.xclbin")

if ret:                                                         
    print("ERROR: Unable to create handle to FPGA")
else:
    print("INFO: Successfully created handle to FPGA")